# Perfect central extensions of $L_3(4)$ in the Monster group

* Author: Thomas Breuer
* Version: OSCAR version 1.0
* Last modified: April 11, 2024

There was [the question in MathOverflow](https://mathoverflow.net/questions/450255)
which perfect central extensions of the simple group $G = L_3(4)$ are subgroups of the sporadic simple Monster group $M$.

The following computations with [OSCAR](https://oscar.computeralgebra.de) answer this question except for one candidate.

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.1.0-DEV


First we get the list of perfect central extensions of $G$ (assuming that their character tables are contained in the character table library).

In [2]:
simp = character_table("L3(4)");
extnames = all_character_table_names(is_perfect => true);
filter!(x -> endswith(x, "L3(4)"), extnames);
ext = map(character_table, extnames);
filter!(x -> length(class_positions_of_center(x)) == order(x) // order(simp), ext);
sort!(ext, by = order);
names = map(identifier, ext)

14-element Vector{String}:
 "L3(4)"
 "2.L3(4)"
 "3.L3(4)"
 "2^2.L3(4)"
 "4_1.L3(4)"
 "4_2.L3(4)"
 "6.L3(4)"
 "(2x4).L3(4)"
 "(2^2x3).L3(4)"
 "12_1.L3(4)"
 "12_2.L3(4)"
 "4^2.L3(4)"
 "(2x12).L3(4)"
 "(4^2x3).L3(4)"

The fact that $G$ is *not* isomorphic to a subgroup of $M$ is shown at the end of the paper [On subgroups of the Monster containing $A_5$'s](http://dx.doi.org/10.1016/j.jalgebra.2003.11.014) by Petra E. Holmes and Robert A. Wilson.

And the following embeddings of central extensions of $G$ in $M$ can be established using known subgroups of $M$.

- $2.G < 2.U_4(3) < 2^2.U_6(2) < Fi_{23} < 3.Fi_{24}' < M$,
- $2^2.G < He < 3.Fi_{24}' < M$,
- $6.G < 2.G_2(4) < 6.Suz < 3^{1+12}_+.2Suz < M$.

Note that $G$ is a subgroup of $U_4(3)$ but not of $2.U_4(3)$, $3.G$ is a subgroup of $G_2(4)$ but not of $2.G_2(4)$, and $G_2(4)$ is a subgroup of $Suz$ but not of $2.Suz$.

The positive statements follow from the ATLAS of Finite Groups (pages 52, 97, 131) and the negative ones from the following computations.


In [3]:
length(possible_class_fusions(character_table("L3(4)"),
                              character_table("2.U4(3)")))

0

In [4]:
length(possible_class_fusions(character_table("3.L3(4)"),
                              character_table("2.G2(4)")))

0

In [5]:
length(possible_class_fusions(character_table("G2(4)"),
                              character_table("2.Suz")))

0

The group $3.G$ centralizes an element of order three.
If $3.G$ is a subgroup of $M$ then it is contained in
a `3A` centralizer (of the structure $3.Fi_{24}'$),
a `3B` centralizer (of the structure $3^{1+12}_+.2Suz$) or
a `3C` centralizer (of the structure $3 \times Th$).
Clearly the case `3C` cannot occur,
and `3B` is excluded by the fact that no class fusion between $3.G$
and the `3B` normalizer $3^{1+12}_+.2Suz.2$ is possible.

In [6]:
t = character_table("MN3B");
println(t)

character table of 3^(1+12).2.Suz.2


In [7]:
length(possible_class_fusions(character_table("3.L3(4)"), t))

0

If $3.G$ is contained in the `3A` centralizer then this embedding induces one of $G$ into some maximal subgroup of
$Fi_{24}'$.
Using the known character tables of these maximal subgroups in GAP's character table library, one shows that only $Fi_{23}$ admits a class fusion, but this subgroup lifts to $3 \times Fi_{23}$ in $3.Fi_{24}'$ and thus cannot lead to a subgroup of type $3.G$.

In [8]:
mx = map(character_table, maxes(character_table("Fi24'")));
s = character_table("L3(4)");
filter(x -> length(possible_class_fusions(s, x)) > 0, mx)

1-element Vector{Oscar.GAPGroupCharacterTable}:
 character table of Fi23

The other candidates $m.G$ contain at least one central involution.
If $m.G$ is a subgroup of $M$ then it is contained in
a `2A` centralizer (of the structure $2.B$) or
a `2B` centralizer (of the structure $2^{1+24}_+.Co_1$).
Again we use `possible_class_fusions` to list all candidates for the class fusion,
but here we prescribe the central involution of the `2A` or `2B`
centralizer as an image of one central involution in $m.G$.

In [9]:
done = ["L3(4)", "2.L3(4)", "3.L3(4)", "2^2.L3(4)", "6.L3(4)"];
filter!(x -> !(x in done), names)

9-element Vector{String}:
 "4_1.L3(4)"
 "4_2.L3(4)"
 "(2x4).L3(4)"
 "(2^2x3).L3(4)"
 "12_1.L3(4)"
 "12_2.L3(4)"
 "4^2.L3(4)"
 "(2x12).L3(4)"
 "(4^2x3).L3(4)"

In [10]:
invcent = map(character_table, ["MN2A", "MN2B"])

2-element Vector{Oscar.GAPGroupCharacterTable}:
 character table of 2.B
 character table of 2^1+24.Co1

In [11]:
map(class_positions_of_center, invcent)

2-element Vector{Vector{Int64}}:
 [1, 2]
 [1, 2]

Computing possible class fusions from all character tables given by `names` to the two involution centralizers would take about an hour of CPU time. In the following, we leave out the group $(2 \times 4).L_3(4)$.

In [12]:
cand = [];
for name in names
  (name == "(2x4).L3(4)") && continue
  s = character_table(name)
  ords = orders_class_representatives(s)
  invpos = filter(i -> ords[i] == 2, class_positions_of_center(s))
  for i in invpos
    for t in invcent
      init = approximate_class_fusion(s, t)
      length(init) == 0 && continue
      init[i] = 2
      fus = possible_class_fusions(s, t, decompose = false, fusionmap = init)
      length(fus) == 0 || push!(cand, [ s, t, i, fus ])
    end
  end
end;
[x[1:3] for x in cand]

1-element Vector{Vector{Any}}:
 [character table of 4_1.L3(4), character table of 2^1+24.Co1, 3]

(Note that we have called `possible_class_fusions` with the option `decompose = false`, in order to save space and time.)

Thus it turns out that exactly one of the checked groups $m.G$ cannot be excluded this way.
Namely, these character-theoretical criteria leave the possibility that $4_1.G$ may occur as a subgroup of $2^{1+24}_+.Co_1$.

Moreover, we see that if this happens then the centre $C$ of $4_1.G$ lies inside the normal subgroup $N = 2^{1+24}_+$.
The centralizer of $C$ in $N$ has order $2^{24}$, and the centralizer of $C$ in $2^{1+24}_+.Co_1$ has order
$2^{24} \cdot |Co_3|$. We see that $4_1.G$, if it exists as a subgroup of $2^{1+24}_+.Co_1$, must lie inside the subgroup $[2^{24}].Co_3$.

In [13]:
s, t, fus = cand[1][1], cand[1][2], cand[1][4]

(character table of 4_1.L3(4), character table of 2^1+24.Co1, [[1, 5, 2, 5, 9, 8, 23, 27, 24, 27  …  71, 74, 114, 119, 115, 119, 114, 119, 115, 119]])

In [14]:
class_positions_of_center(s)

4-element Vector{Int64}:
 1
 2
 3
 4

In [15]:
5 in class_positions_of_pcore(t, 2)

true

In [16]:
siz, _ = divrem( orders_centralizers(t)[5], 2^24 )

(495766656000, 0)

In [17]:
mx = filter(x -> mod(order(x), siz) == 0,
            [character_table(x) for x in maxes(character_table("Co1"))])

1-element Vector{Oscar.GAPGroupCharacterTable}:
 character table of Co3

In [18]:
order(mx[1]) == siz

true

I do not see a character-theoretic argument that could disprove the existence of such a $4_1.G$ type subgroup.

Concerning the candidate $H = (2 \times 4).L_3(4)$, it is enough to show that no fusions to the involution centralizers $C$ exist that map the central involution in class $4$ of $H$ to the central involution of $C$.
This proves that $H$ does not embed into the Monster group.

In [19]:
s = character_table("(2x4).L3(4)"); 4 in class_positions_of_center(s)

true

In [20]:
t = invcent[1]; init = approximate_class_fusion(s, t); init[4] = 2;
length(possible_class_fusions(s, t, decompose = false, fusionmap = init))

0

In [21]:
t = invcent[2]; init = approximate_class_fusion(s, t); init[4] = 2;
# The following computation needs about 10 minutes.  We leave it out.
# length(possible_class_fusions(s, t, decompose = false, fusionmap = init))

0